# Notebook Setup

In [1]:
if 'google.colab' in str(get_ipython()):
  IN_COLLAB = True
else:
  IN_COLLAB = False

#TODO: CHANGE THIS BASED ON YOUR OWN LOCAL SETTINGS
MY_HOME_ABS_PATH = "/content/drive/MyDrive/W210/co2-flux-hourly-gpp-modeling"

if IN_COLLAB:
  from google.colab import drive
  drive.mount('/content/drive/')

Mounted at /content/drive/


## Import Modules

In [2]:
# install required modules quietly
required_packages = ['azure-storage-blob']

for p in required_packages: 
  try:
      __import__(p)
  except ImportError:
      %pip install {p} --quiet

import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
import pandas as pd
import numpy as np
import sys

# Load locale custome modules
os.chdir(MY_HOME_ABS_PATH)
if IN_COLLAB:
  sys.path.insert(0,os.path.abspath("./code/src/tools"))
else:
  sys.path.append(os.path.abspath("./code/src/tools"))

from CloudIO.AzStorageClient import AzStorageClient
from data_pipeline_lib import *

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 387.8/387.8 KB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.5/174.5 KB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 KB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 46.1 MB/s eta 0:00:00


# Define Constants

In [3]:
root_dir =  MY_HOME_ABS_PATH
tmp_dir =  root_dir + os.sep + '.tmp'
raw_data_dir = tmp_dir
data_dir = root_dir + os.sep + 'data'
cred_dir = root_dir + os.sep + '.cred'
az_cred_file = cred_dir + os.sep + 'azblobcred.json'

if IN_COLLAB:
  raw_data_dir = "/content/drive/MyDrive/CO2_flux_gpp_modeling/DS_capstone_23Spring_CO2/Data/half_hourly_data"

site_metadata_filename = data_dir + os.sep + 'site-metadata.csv'
monthly_data_filename = data_dir + os.sep + "monthly-imputed-v1-i.csv"

# File
container = "all-sites-data"
ext = "parquet"
ver = "exp1"
blob_name_base = f"full_2010_2015_all_v_{ver}"
train_blob_name = f"full_2010_2015-train-v-{ver}.{ext}"
test_blob_name = f"full_2010_2015-test-v-{ver}.{ext}"

In [4]:
# Define features and target variables of the data pipelines
target_variable = 'GPP_NT_VUT_REF'
target_variable_qc = 'NEE_VUT_REF_QC'
hourly_features = ['TA_ERA', 'SW_IN_ERA', 'LW_IN_ERA', 'VPD_ERA', 'P_ERA', 'PA_ERA',
                     'datetime', 'year', 'month', 'day', 'hour', 'date',
                     'EVI', 'NDVI', 'NIRv', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7']
metadata_features = ['site_id', 'filename', 'lat', 'long', 'koppen_sub', 'koppen_main', 'IGBP',
                     'c3c4', 'c4_percent', 'monthly_data_available']

# Define the features to use in KNN imputer, only using real values as cat are same per site
imp_exclude_cols = ['date', 'datetime', 'year', 'month', 'hour', 'day', 'minute', 'site_id', 'IGBP']
imp_cols = [x for x in hourly_features + ['GPP_NT_VUT_REF'] if x not in imp_exclude_cols]

In [5]:
site_splits =[
  ['AR-SLu', 'AU-ASM', 'AU-Cum', 'AU-How', 'CA-TP3', 'CA-TPD', 'CN-Sw2', 'DE-Lnf',
   'IT-CA3', 'NL-Hor', 'US-Syv', 'US-AR2', 'US-ARM', 'US-Vcp', 'CH-Cha', 'CZ-KrP', 
   'CZ-Lnz', 'DE-Geb', 'DE-Obe', 'ES-LJu', 'FI-Let', 'IT-Lav', 'SE-Deg'],
  ['AU-Cpr', 'AU-Wom', 'CZ-BK2', 'DE-SfN', 'IT-CA1', 'IT-CA2', 'IT-Ro2', 'US-IB2', 
   'US-Me6', 'US-Ton', 'CA-Ca3', 'US-CRT', 'US-KFS', 'US-Mpj', 'US-Prr', 'US-Ro1', 
   'US-Tw4', 'BE-Bra', 'CZ-BK1', 'DE-Hai', 'IL-Yat', 'IT-Tor', 'SE-Htm'],
  ['AT-Neu', 'AU-RDF', 'AU-Whr', 'CA-TP1', 'DE-Zrk', 'IT-SRo', 'US-Wkg', 'CA-Ca2',
   'CA-TP4', 'US-Bar', 'US-Fmf', 'US-Oho', 'US-SRG', 'US-Ses', 'CH-Lae', 'CZ-RAJ',
   'CZ-wet', 'DE-Kli', 'DE-RuR', 'ES-LM2', 'FR-Fon', 'FR-Lam'],
  ['AR-Vir', 'AU-DaS', 'AU-Emr', 'AU-Gin', 'AU-Rig', 'AU-TTE', 'DE-Spw', 'FR-Pue',
   'IT-Isp', 'IT-Noe', 'US-Twt', 'US-WPT', 'CA-Cbo', 'US-Vcm', 'BE-Dor', 'BE-Vie',
   'CZ-Stn', 'FI-Hyy', 'SE-Nor', 'SE-Ros', 'NL-Loo', 'SE-Lnn'],
  ['AU-DaP', 'AU-GWW', 'AU-Rob', 'AU-Stp', 'US-GLE', 'US-NR1', 'US-Whs', 'CA-Ca1',
   'CA-Gro', 'US-AR1', 'US-Rws', 'US-UMd', 'US-Wjs', 'CH-Fru', 'CH-Oe2', 'DE-Tha',
   'DK-Sor', 'FR-Bil', 'FR-Hes', 'IT-BCi', 'IT-SR2', 'DE-Hte'],
  ['CA-Oas', 'ES-Amo', 'FI-Sod', 'US-Myb', 'US-SRM', 'US-Tw3', 'US-Var', 'US-WCr',
   'US-Ho1', 'US-Seg', 'US-UMB', 'BE-Lon', 'CH-Dav', 'DE-Gri', 'DE-HoH', 'ES-LM1',
   'FR-Aur', 'FR-FBn', 'GF-Guy', 'IT-MBo', 'IT-Ren', 'RU-Fyo']
]

# Stage 1: Trim and Merge Site Metadata

In [15]:
# Define imput params
impute = True
impute_method = 'knn'
impute_global = True
resample = True
time_col = 'datetime'
duration = 'H'

# Filter sequence to date range
missing_thresh = 0.2
start_date = '2010-01-01'
end_date ='2015-12-31'

# Impute params (if used)
k=5
weights='uniform'
n_fit=20000
c=-1

train_sites = [item for sublist in site_splits[:3] for item in sublist] 
test_sites = site_splits[:1]
print(len(train_sites))
print(len(test_sites))

68
1


In [16]:
prep_hourly = PrepareAllSitesHourly(site_metadata_filename, monthly_data_filename, train_sites, test_sites, 
                                    hourly_features, metadata_features, target_variable_qc, target_variable, raw_data_dir)

data_df = prep_hourly.all_sites_all_sources(imp_cols, resample, impute, impute_method, impute_global,
                                            k, weights, n_fit, time_col, duration, start_date, end_date, missing_thresh, c)

1it [00:03,  3.59s/it]

Processing: 1. AR-SLu
Processing: 2. AT-Neu


1it [00:29, 29.16s/it]


KeyboardInterrupt: ignored

In [ ]:
# Get the memory usage of the dataframe in bytes
memory_usage = data_df.memory_usage().sum()
memory_usage_gb = memory_usage / 1_000_000_000
print(f"The dataframe uses {memory_usage_gb:.2f} GB of memory.")

In [ ]:
data_df['site_id'].unique()

# CHECKPOINT: Save full raw data

In [ ]:
# Upload to Azure Storage Blob
# ref: https://stackoverflow.com/a/54666079
from io import BytesIO
data_cleanup_checkpoint = True
tag = "raw"
blob_name = f"{blob_name_base}_{tag}.{ext}"

if data_cleanup_checkpoint:

  parquet_file = BytesIO()
  data_df.to_parquet(parquet_file, engine='pyarrow')
  parquet_file.seek(0)

  azStorageClient = AzStorageClient(az_cred_file)
  azStorageClient.uploadBlob(container, blob_name, parquet_file, overwrite=True)

File uploaded to baseline-data/baseline_all_v_1-i-knn_raw.parquet


# Stage 2: Data Transform - Convert to Model Ready Data
- Encode data
- Split into train and test
- Assemble data through VectorAssembler
- Normalized data through minmax

In [ ]:
raw_data_file_path = None
data_transformer = TFTDataTransformer(train_sites, test_sites, raw_data_file_path, data_df)

Data size: (865920, 49).


In [ ]:
non_transform_cols = [target_variable, 'site_id', 'datetime', 'timestep_idx_local', 'timestep_idx_global', 'gap_flag_hour', 'gap_flag_month']
categorical_cols = ['IGBP', 'c3c4', 'koppen_sub', 'koppen_main',
                    'year', 'month', 'day', 'hour', 'MODIS_PFT', 'MODIS_LC']
realNum_cols = ['TA_ERA', 'SW_IN_ERA', 'LW_IN_ERA', 'VPD_ERA', 'P_ERA', 'PA_ERA', 
                'EVI', 'NDVI', 'NIRv', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7',
                'lat', 'long', 'c4_percent',
                'BESS-PAR', 'BESS-PARdiff', 'BESS-RSDN', 'CSIF-SIFdaily',
                'PET', 'Ts', 'ESACCI-sm', 'NDWI', 'Percent_Snow', 'Fpar', 'Lai',
                'LST_Day', 'LST_Night']
data_transformer.data_transform(categorical_cols, realNum_cols, non_transform_cols)

print("\nTrain data peak:")
display(data_transformer.train_df.head(5))
print("\nTest data peak:")
display(data_transformer.test_df.head(5))

Data size: (865920, 49).
Data size after encoding: (865920, 49)
Number of sites in df: 19
Train Sites: ['IT-Lav', 'US-NR1', 'US-Vcp', 'FR-Pue', 'CH-Lae', 'US-Var', 'US-Ne2', 'ES-LJu', 'US-Ton', 'US-UMB', 'US-Me2', 'FI-Hyy', 'US-NR1', 'IT-Lav', 'US-Wkg', 'US-ARM', 'US-SRM']
Test Sites: ['US-GLE', 'US-AR1', 'US-Seg', 'US-FR2', 'ES-LM2', 'CA-Cbo', 'FR-Lam', 'IT-Cpz', 'CN-Cha', 'IT-Lsn']
Normalizing real features (32)
Train data size: (631032, 49).
Test data size: (234888, 49).

Train data peak:


,GPP_NT_VUT_REF,site_id,timestep_idx_local,timestep_idx_global,datetime,year,month,day,hour,TA_ERA,SW_IN_ERA,LW_IN_ERA,VPD_ERA,P_ERA,PA_ERA,EVI,NDVI,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,lat,long,koppen_sub,koppen_main,c3c4,c4_percent,BESS-PAR,BESS-PARdiff,BESS-RSDN,CSIF-SIFdaily,PET,Ts,ESACCI-sm,MODIS_LC,NDWI,Percent_Snow,Fpar,Lai,LST_Day,LST_Night,MODIS_PFT,gap_flag_hour,gap_flag_month
0,8.00070,CH-Lae,0,0,2010-01-01 00:00:00,0,0,0,0,-0.93215,-0.71272,-1.06375,-0.86296,-0.20802,0.05816,0.38113,0.83348,0.36236,-0.78665,-0.43050,-0.59344,-0.84229,-0.51919,-0.62349,-0.75516,5,0.77039,1.18369,5,2,0,-0.58974,-1.62789,-1.41854,-1.63492,-0.86451,1.08868,-1.48659,0.93231,5,0.15866,2.32445,-1.47498,-0.96933,-1.47888,-1.10114,5,0.00000,0.00000
1,8.04417,CH-Lae,1,1,2010-01-01 01:00:00,0,0,0,1,-0.96828,-0.71272,-1.07565,-0.86149,-0.20802,0.05344,0.38113,0.83348,0.36236,-0.78665,-0.43050,-0.59344,-0.84229,-0.51919,-0.62349,-0.75516,5,0.77039,1.18369,5,2,0,-0.58974,-1.62789,-1.41854,-1.63492,-0.86451,1.08868,-1.48659,0.93231,5,0.15866,2.32445,-1.47498,-0.96933,-1.47888,-1.10114,5,0.00000,0.00000
2,7.96453,CH-Lae,2,2,2010-01-01 02:00:00,0,0,0,2,-0.97002,-0.71272,-1.17342,-0.85560,-0.20802,0.04883,0.38113,0.83348,0.36236,-0.78665,-0.43050,-0.59344,-0.84229,-0.51919,-0.62349,-0.75516,5,0.77039,1.18369,5,2,0,-0.58974,-1.62789,-1.41854,-1.63492,-0.86451,1.08868,-1.48659,0.93231,5,0.15866,2.32445,-1.47498,-0.96933,-1.47888,-1.10114,5,0.00000,0.00000
3,12.28330,CH-Lae,3,3,2010-01-01 03:00:00,0,0,0,3,-1.01993,-0.71272,-1.09669,-0.84749,-0.20802,0.04701,0.38113,0.83348,0.36236,-0.78665,-0.43050,-0.59344,-0.84229,-0.51919,-0.62349,-0.75516,5,0.77039,1.18369,5,2,0,-0.58974,-1.62789,-1.41854,-1.63492,-0.86451,1.08868,-1.48659,0.93231,5,0.15866,2.32445,-1.47498,-0.96933,-1.47888,-1.10114,5,0.00000,0.00000
4,13.75710,CH-Lae,4,4,2010-01-01 04:00:00,0,0,0,4,-1.05280,-0.71272,-1.02725,-0.85290,-0.20802,0.04121,0.38113,0.83348,0.36236,-0.78665,-0.43050,-0.59344,-0.84229,-0.51919,-0.62349,-0.75516,5,0.77039,1.18369,5,2,0,-0.58974,-1.62789,-1.41854,-1.63492,-0.86451,1.08868,-1.48659,0.93231,5,0.15866,2.32445,-1.47498,-0.96933,-1.47888,-1.10114,5,0.00000,0.00000



Test data peak:


,GPP_NT_VUT_REF,site_id,timestep_idx_local,timestep_idx_global,datetime,year,month,day,hour,TA_ERA,SW_IN_ERA,LW_IN_ERA,VPD_ERA,P_ERA,PA_ERA,EVI,NDVI,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,lat,long,koppen_sub,koppen_main,c3c4,c4_percent,BESS-PAR,BESS-PARdiff,BESS-RSDN,CSIF-SIFdaily,PET,Ts,ESACCI-sm,MODIS_LC,NDWI,Percent_Snow,Fpar,Lai,LST_Day,LST_Night,MODIS_PFT,gap_flag_hour,gap_flag_month
0,1.30673,CA-Cbo,0,0,2010-01-01 00:00:00,0,0,0,0,-1.17580,-0.71272,0.22907,-0.87683,-0.07995,0.87251,1.34874,-0.72758,-0.22281,1.01824,1.03660,2.44975,1.58852,-0.77073,-1.35122,-1.05468,1,0.36830,-0.34851,5,2,0,-0.52345,-1.14726,-0.63639,-1.16041,-1.01060,0.92265,-1.63887,-0.85179,5,0.49786,3.35380,-0.53467,-0.57577,-1.89649,-2.01633,5,0.00000,1.00000
1,1.38098,CA-Cbo,1,1,2010-01-01 01:00:00,0,0,0,1,-1.18091,-0.71272,0.11045,-0.87659,-0.06518,0.87112,0.82468,-0.72760,-0.47349,1.25023,0.82720,2.63323,1.93407,-0.50555,-1.24262,-0.97915,1,0.36830,-0.34851,5,2,0,-0.52345,-1.14726,-0.63639,-1.16041,-1.01060,0.92265,-1.63887,-0.85179,5,0.49786,3.35380,-0.53467,-0.57577,-1.89649,-2.01633,5,0.00000,1.00000
2,0.27869,CA-Cbo,2,2,2010-01-01 02:00:00,0,0,0,2,-1.19918,-0.71272,0.11045,-0.87389,-0.06518,0.87198,0.82468,-0.72760,-0.47349,1.25023,0.82720,2.63323,1.93407,-0.50555,-1.24262,-0.97915,1,0.36830,-0.34851,5,2,0,-0.52345,-1.14726,-0.63639,-1.16041,-1.01060,0.92265,-1.63887,-0.85179,5,0.49786,3.35380,-0.53467,-0.57577,-1.89649,-2.01633,5,0.00000,1.00000
3,-1.72097,CA-Cbo,3,3,2010-01-01 03:00:00,0,0,0,3,-1.21745,-0.71272,0.11045,-0.87119,-0.06518,0.87284,0.65217,-1.17147,-0.60127,1.76995,1.27795,3.17049,2.50126,-0.18988,-1.10024,-0.95277,1,0.36830,-0.34851,5,2,0,-0.52345,-1.14726,-0.63639,-1.16041,-1.01060,0.92265,-1.63887,-0.85179,5,0.49786,3.35380,-0.53467,-0.57577,-1.89649,-2.01633,5,0.00000,1.00000
4,0.67984,CA-Cbo,4,4,2010-01-01 04:00:00,0,0,0,4,-1.23572,-0.71272,0.00028,-0.86849,0.12200,0.87369,0.95731,-0.72295,-0.32818,1.09781,0.93626,2.29454,1.57462,-0.74375,-1.12675,-0.88474,1,0.36830,-0.34851,5,2,0,-0.52345,-1.14726,-0.63639,-1.16041,-1.01060,0.92265,-1.63887,-0.85179,5,0.49786,3.35380,-0.53467,-0.57577,-1.89649,-2.01633,5,0.00000,1.00000


# FINISH: Upload train and test to Azure Blob Storage

In [ ]:
final_checkpoint = True

if final_checkpoint:
  data_transformer.upload_train_test_to_azure(az_cred_file, container, train_blob_name, test_blob_name)

Uploading train dataset to baseline-train-v-1-i-knn.parquet...
File uploaded to baseline-data/baseline-train-v-1-i-knn.parquet
Uploading test dataset to baseline-test-v-1-i-knn.parquet...
File uploaded to baseline-data/baseline-test-v-1-i-knn.parquet
